Twelve Labs Video Understanding Platform, currently in beta, offers an API suite for integrating a state-of-the-art (“SOTA”) foundation model that understands contextual information from your videos, making it accessible to your applications. The API is organized around REST and is compatible with most programming languages. You can also use Postman or other REST clients to send requests and view responses.

The following diagram illustrates the architecture of the Twelve Labs Video Understanding Platform and how different parts interact:

![](https://files.readme.io/5fb7a80-image.png)

## 1 - Setup and Requirements
The Twelve Labs Video Understanding API is currently in open beta. Before you begin, [sign up](https://api.twelvelabs.io) for a free account, or if you already have one, [sign in](https://api.twelvelabs.io).

You will receive a signup code that you can use to create an account. Once you have created an account, you can access the API Dashboard.

This demo will use an existing account. To make calls to the API, you need your secret key and specify the URL of the API. You can use environment variables to pass configuration to your application.

In [25]:
# Specify the API key and the URL of the API
%env API_KEY=tlk_3Q0CS7N2TKJJK42D1DQQ71SWFX3S
%env API_URL=https://api.twelvelabs.io/v1.2

env: API_KEY=tlk_3Q0CS7N2TKJJK42D1DQQ71SWFX3S
env: API_URL=https://api.twelvelabs.io/v1.2


The next step is to install the required package.

In [26]:
!pip install requests

You want to import the required Python packages and retrieve the API key as well as the API URL.

In [27]:
# Import the required packages
import requests
import glob
from pprint import pprint
import os

In [28]:
# Retrieve the URL of the API and my API key
API_URL = os.getenv("API_URL")
assert API_URL

API_KEY = os.getenv("API_KEY")
assert API_KEY

## 2 - Index API
The next step is to [create a video index](https://docs.twelvelabs.io/v1.2/docs/create-indexes). An index groups one or more video vectors and is the most granular level at which you can perform a search.

### 2.1 - Create An Index
An index is a basic unit to organize and store your video data (video embeddings and metadata) to facilitate information retrieval and processing.

You can use indexes to group related videos. For example, if you want to upload multiple videos from a car race, Twelve Labs recommends you create a single index and upload all the videos to it. Then, to find a specific moment in that race, you call the /search endpoint once, passing it the unique identifier of the index.

When creating a new index, you must specify the following information:

*   **Name**: Use a brief and descriptive name to facilitate future reference and management.
*   **Engine configuration**: Provide a list containing the [video understanding engines](https://docs.twelvelabs.io/v1.2/docs/video-understanding-engines) and the associated [engine options](https://docs.twelvelabs.io/v1.2/docs/indexing-options) you want to enable.

#### Video Understanding Engines
Twelve Labs' video understanding engines consist of a family of deep neural networks built on our multimodal foundation model for video understanding that you can use for the following downstream tasks:

*   Search using natural language queries
*   Zero-shot classification
*   Generate concise textual representations

Twelve Labs provides two distinct engine types - embedding and generative, each serving unique purposes in multimodal video understanding.
*   **Embedding engines**: These engines are proficient at performing tasks such as search and classification, enabling enhanced video understanding. We recommend choosing Marengo2.5.
*   **Generative engines**: These engines specialize in generating concise textual representations for video. We recommend choosing Pegasus1.0.

#### Engine Options

Engine options indicate the types of information a video understanding engine will process. When creating a new index, you must specify the engines and the associated engines options that you want to enable.

The following engine options are available:

*   **visual**: This option allows you to analyze video content as you would see and hear from it, including actions, objects, sounds, and events, and excluding human speech.
*   **conversation**: This option allows you to analyze human speech within your videos.
*   **text_in_video**: This option allows you to detect and extract text (OCR) shown within your videos.
*   **logo**: This option allows you to identify the presence of brand logos within your videos.

In [29]:
# Import the required packages
import requests
import glob
from pprint import pprint
import os

# Retrieve the URL of the API and my API key
API_URL = os.getenv("API_URL")
assert API_URL

API_KEY = os.getenv("API_KEY")
assert API_KEY

# Construct the URL of the `/indexes` endpoint
INDEXES_URL = f"{API_URL}/indexes"

print(f"API_URL: {API_URL}")  # Print the API base URL
print(f"INDEXES_URL: {INDEXES_URL}")

# Specify the name of the index
INDEX_NAME = "Content Creator Videos"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named data
data = {
  "engines": [
    {
      "engine_name": "marengo2.6",  # Configure the video understanding engine
      "engine_options": ["visual", "conversation", "text_in_video", "logo"]  # Specify the engine options
    },
    {
      "engine_name": "pegasus1",  # Configure the video understanding engine
      "engine_options": ["visual", "conversation"]  # Specify the engine options
    }
  ],
  "index_name": INDEX_NAME,
  "addons": ["thumbnail"]  # Enable the thumbnail generation feature
}

# Create an index
response = requests.post(INDEXES_URL, headers=headers, json=data)

# Check for success before attempting to decode JSON
if response.status_code == 200:
    # Store the unique identifier of your index
    INDEX_ID = response.json().get('_id')
    print(f'Index created with ID: {INDEX_ID}')
else:
    # Print the status code and full response content for debugging
    print(f'Error creating index. Status code: {response.status_code}')
    print(response.text)  # Print the raw response content

# Print the status code and response
#print(f'Status code: {response.status_code}')
#pprint(response.json())

API_URL: https://api.twelvelabs.io/v1.2
INDEXES_URL: https://api.twelvelabs.io/v1.2/indexes
Error creating index. Status code: 201
{"_id":"666599d4e6fb7df29de0f89c"}



### 2.2 - Upload A Video
Once you've created an index, you can upload a video. When the video has finished uploading, the API service will automatically create a rich set of vectors containing all the information it needs to perform fast, semantic, accurate, and scalable searches.

For this demo, we will work with sports videos. The video resolution must be greater or equal than 360p and less or equal than 4K.

Below, we show how to upload a video file from YouTube: [Jenna Marbles - How Girls Get Dressed](https://youtu.be/0Hc0Z6zFbzM?si=a1oqVb0Eve9yzxMD)

The API service will retrieve the file directly from the specified URL, so your application doesn't have to store the video locally and upload the bytes. Our platform currently supports only YouTube as an external provider, but we will add support for additional providers in the future.

In [32]:
# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare the /tasks/external-provider endpoint
TASKS_URL = f"{API_URL}/tasks/external-provider"

# Construct the body of the request
data = {
    "index_id": "666599d4e6fb7df29de0f89c",  # Specify the unique ID of the index
    "url": "https://www.youtube.com/watch?v=73_1biulkYk.mp4",  # Specify the YouTube URL of the video
}

# Upload the video
response = requests.post(TASKS_URL, headers=headers, json=data)

# Store the ID of the task in a variable named TASK_ID
TASK_ID = response.json().get("_id")

# Print the status code and response
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 201
{'_id': '66659a3efa7835326810120c'}


The API service must have finished indexing the video before it becomes searchable. You can use the `GET` method of the `/tasks/{_id}` endpoint to monitor the indexing process. Construct the URL for monitoring the indexing process based on the `TASK_ID` variable you’ve declared in the previous step, and wait until the status shows as `ready`:

In [33]:
import time

# Define starting time
start = time.time()
print("Start uploading video")

# Monitor the indexing process
TASK_STATUS_URL = f"{API_URL}/tasks/{TASK_ID}"
while True:
    response = requests.get(TASK_STATUS_URL, headers=headers)
    STATUS = response.json().get("status")
    if STATUS == "ready":
        print(f"Status code: {STATUS}")
        break
    time.sleep(10)

# Define ending time
end = time.time()
print("Finish uploading video")
print("Time elapsed (in seconds): ", end - start)

Start uploading video
Status code: ready
Finish uploading video
Time elapsed (in seconds):  245.4978461265564


In [34]:
# Retrieve the unique identifier of the video
VIDEO_ID = response.json().get('video_id')

# Print the status code, the unique identifier of the video, and the response
print(f"VIDEO ID: {VIDEO_ID}")
pprint(response.json())

VIDEO ID: 66659a48d22b3a3c97bf1d5f
{'_id': '66659a3efa7835326810120c',
 'created_at': '2024-06-09T12:04:14.069Z',
 'estimated_time': '2024-06-09T12:07:40.728Z',
 'hls': {'status': 'COMPLETE',
         'thumbnail_urls': ['https://deuqpmn4rs7j5.cloudfront.net/66649f09937ce44a0402bad4/66659a3efa7835326810120c/thumbnails/fd31bdd0-eaf4-427f-bc27-6cb7939e6249.0000001.jpg'],
         'updated_at': '2024-06-09T12:04:59.781Z',
         'video_url': 'https://deuqpmn4rs7j5.cloudfront.net/66649f09937ce44a0402bad4/66659a3efa7835326810120c/stream/fd31bdd0-eaf4-427f-bc27-6cb7939e6249.m3u8'},
 'index_id': '666599d4e6fb7df29de0f89c',
 'metadata': {'duration': 159,
              'filename': 'Deadpool & Wolverine | Official Trailer | In '
                          'Theaters July 26',
              'height': 720,
              'width': 1280},
 'status': 'ready',
 'type': 'index_task_info',
 'updated_at': '2024-06-09T12:08:52.222Z',
 'video_id': '66659a48d22b3a3c97bf1d5f'}


Note that the 1.2 version of the API is in beta, and videos can take longer to upload and process when using the Pegasus video understanding engine.

For the purpose of this demo, let's use a sample account with sample indexes that has been preloaded with videos. These indexes have been processed with both the Marengo 2.5 and the Pegasus 1.0 engines.

In [35]:
# Specify the API key and the API URL of the sample account
%env API_KEY=tlk_3Q0CS7N2TKJJK42D1DQQ71SWFX3S
%env API_URL=https://api.twelvelabs.io/v1.2

env: API_KEY=tlk_3Q0CS7N2TKJJK42D1DQQ71SWFX3S
env: API_URL=https://api.twelvelabs.io/v1.2


In [36]:
# Retrieve the new API URL and the new API key
API_URL = os.getenv("API_URL")
assert API_URL

API_KEY = os.getenv("API_KEY")
assert API_KEY

### 2.3 - List Indexes
Let's return a list of the indexes in this account. The API returns indexes sorted by creation date, with the oldest indexes at the top of the list.

In [38]:
# Set the header of the request
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# List indexes
url = f"{API_URL}/indexes?page=1&page_limit=10&sort_by=created_at&sort_option=asc"

response = requests.get(url, headers=headers)
print(f'Status code: {response.status_code}')
pprint(response.json())

Status code: 200
{'data': [{'_id': '66649f31937ce44a0402bad8',
           'addons': ['thumbnail'],
           'created_at': '2024-06-08T18:13:05.563Z',
           'engines': [{'engine_name': 'marengo2.6',
                        'engine_options': ['visual',
                                           'conversation',
                                           'text_in_video',
                                           'logo']},
                       {'engine_name': 'pegasus1',
                        'engine_options': ['visual', 'conversation']}],
           'expires_at': '2024-09-06T18:13:05.563Z',
           'index_name': 'My Index (Default)',
           'total_duration': 0,
           'updated_at': '2024-06-08T18:13:05.563Z',
           'video_count': 0},
          {'_id': '66653a83fa7835326810108d',
           'addons': ['thumbnail'],
           'created_at': '2024-06-09T05:15:47.545Z',
           'engines': [{'engine_name': 'marengo2.6',
                        'engine_options': ['

For this demo, let's go with Content Creator videos.

In [39]:
%env INDEX_ID=666599d4e6fb7df29de0f89c

env: INDEX_ID=666599d4e6fb7df29de0f89c


### 2.4 - List Videos In An Index
Let's list all the videos in this existing "Content Creator" index for verification.

In [40]:
# Retrieve the unique identifier of the existing index
INDEX_ID = os.getenv("INDEX_ID")
print (INDEX_ID)
assert INDEX_ID

# Set the header of the request
headers = {
    "x-api-key": API_KEY,
    "Content-Type": "application/json"
}

# List all the videos in an index
INDEXES_VIDEOS_URL = f"{API_URL}/indexes/{INDEX_ID}/videos"

response = requests.get(INDEXES_VIDEOS_URL, headers=headers)
print(f'Status code: {response.status_code}')
pprint(response.json())

666599d4e6fb7df29de0f89c
Status code: 200
{'data': [{'_id': '66659a48d22b3a3c97bf1d5f',
           'created_at': '2024-06-09T12:04:14Z',
           'indexed_at': '2024-06-09T12:08:52Z',
           'metadata': {'duration': 159,
                        'engine_ids': ['marengo2.6', 'pegasus1'],
                        'filename': 'Deadpool & Wolverine | Official Trailer | '
                                    'In Theaters July 26',
                        'fps': 24,
                        'height': 720,
                        'size': 10016501,
                        'width': 1280},
           'updated_at': '2024-06-09T12:04:24Z'}],
 'page_info': {'limit_per_page': 10,
               'page': 1,
               'total_duration': 159,
               'total_page': 1,
               'total_results': 1}}


## 3 - Search API
Twelve Labs provides a holistic understanding of your videos, moving beyond the limitations of relying solely on individual types of data like keywords, metadata, or transcriptions. By simultaneously integrating all available forms of information, including images, sounds, spoken words, and on-screen text, the platform captures the complex relationships among these elements for a more human-like interpretation.

Search options specify the source of information the engine uses when performing a search. The following values are supported:

*   **visual**: This option allows you to analyze video content as you would see and hear from it, including actions, objects, sounds, and events, and excluding human speech. The following examples are valid search terms: "birds flying near a castle", "sun shining on water", "chickens on the road", "an officer holding a child's hand.", "crowd cheering in the stadium."
*   **conversation**: This option allows you to analyze human speech within your videos.
*   **text_in_video**: This option allows you to detect and extract text (OCR) shown within your videos.
*   **logo**: This option allows you to identify the presence of brand logos within your videos.

### 3.1 - Visual
The platform allows you to analyze video content as you would see and hear from it, including actions, objects, sounds, and events, excluding human speech. The following example sets the value of the `search_options` parameter to  `["visual"]` to search for "**A woman wearing makeup**" using visual cues:

In [41]:
# Construct the URL of the `/search` endpoint
SEARCH_URL = f"{API_URL}/search/"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

query = "A woman wearing makeup"

# Declare a dictionary named `data`
data = {
    "query": query,  # Specify your search query
    "index_id": INDEX_ID,  # Indicate the unique identifier of your index
    "search_options": ["visual"],  # Specify the search options
}

# Make a search request
response = requests.post(SEARCH_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 10,
               'page_expired_at': '2024-06-09T12:09:42Z',
               'total_inner_matches': 0,
               'total_results': 0},
 'search_pool': {'index_id': '666599d4e6fb7df29de0f89c',
                 'total_count': 1,
                 'total_duration': 159}}


In [42]:
# Import libraries to display visual in the notebook
from IPython.display import display
from IPython.display import Image

# Display a list of thumbnail URLs of the returning videos
res = response.json()
print ("Search Query: ", query)
print ("Option:", "Visual")

for item in res["data"]:
  display(Image(url=item["thumbnail_url"]))
  print("Confidence Level:", item["confidence"])
  print("Confidence Score:", item["score"])

Search Query:  A woman wearing makeup
Option: Visual


### 3.2 - Conversation
The platform allows you to analyze human speech within your videos.
The following example sets the value of the `search_options` parameter to `["conversation"]` to perform a semantic search and returns the matches for the specified search term - "**Strapped in the ocean**":

In [64]:
# Construct the URL of the `/search` endpoint
SEARCH_URL = f"{API_URL}/search/"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

query = "guy with red mask"

# Declare a dictionary named `data`
data = {
    "query": query,  # Specify your search query
    "index_id": INDEX_ID,  # Indicate the unique identifier of your index
    "search_options": ["conversation"],  # Specify the search options
    "conversation_option": "semantic",  # Specifty the conversation option
}

# Make a search request
response = requests.post(SEARCH_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 10,
               'page_expired_at': '2024-06-09T12:17:14Z',
               'total_inner_matches': 0,
               'total_results': 0},
 'search_pool': {'index_id': '666599d4e6fb7df29de0f89c',
                 'total_count': 1,
                 'total_duration': 159}}


In [65]:
# Display a list of thumbnail URLs of the returning videos
res = response.json()
print ("Search Query: ", query)
print ("Option:", "Conversation")

for item in res["data"]:
  print ("GT Text: ", item["metadata"][0]["text"])
  display(Image(url=item["thumbnail_url"]))
  print("Confidence Level:", item["confidence"])
  print("Confidence Score:", item["score"])

Search Query:  guy with red mask
Option: Conversation


### 3.3 - Text In Video
The platform allows you to detect and extract text (OCR) shown within your videos. The following example sets the value of the `search_options` parameter to `["text_in_video"]` to search for text that matches "**Libya**":

In [45]:
# Construct the URL of the `/search` endpoint
SEARCH_URL = f"{API_URL}/search/"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

query = "Libya"

# Declare a dictionary named `data`
data = {
    "query": query,  # Specify your search query
    "index_id": INDEX_ID,  # Indicate the unique identifier of your index
    "search_options": ["text_in_video"],  # Specify the search options
}

# Make a search request
response = requests.post(SEARCH_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 10,
               'page_expired_at': '2024-06-09T12:09:45Z',
               'total_inner_matches': 0,
               'total_results': 0},
 'search_pool': {'index_id': '666599d4e6fb7df29de0f89c',
                 'total_count': 1,
                 'total_duration': 159}}


In [46]:
# Display a list of thumbnail URLs of the returning videos
res = response.json()
print ("Search Query: ", query)
print ("Option:", "Text In Video")

for item in res["data"]:
  display(Image(url=item["thumbnail_url"]))
  print("Confidence Level:", item["confidence"])
  print("Confidence Score:", item["score"])

Search Query:  Libya
Option: Text In Video


### 3.4 - Logo
The platform allows you to detect and extract brand logos as shown within your videos. For this, call the [`POST`](/reference/make-search-request) method of the `/search` endpoint with the following parameters:

- `query`:  The name of the company
- `search_options`:  The source of information the platform uses (`logo`)
- `index_id`: The unique identifier of the index you've previously created

The example code below finds when the Subway logo appears in your videos:

In [47]:
# Construct the URL of the `/search` endpoint
SEARCH_URL = f"{API_URL}/search/"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

query = "Subway"

# Declare a dictionary named `data`
data = {
    "query": query,  # Specify your search query
    "index_id": INDEX_ID,  # Indicate the unique identifier of your index
    "search_options": ["logo"],  # Specify the search options
}

# Make a search request
response = requests.post(SEARCH_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 10,
               'page_expired_at': '2024-06-09T12:09:46Z',
               'total_inner_matches': 0,
               'total_results': 0},
 'search_pool': {'index_id': '666599d4e6fb7df29de0f89c',
                 'total_count': 1,
                 'total_duration': 159}}


In [48]:
# Display a list of thumbnail URLs of the returning videos
res = response.json()
print ("Search Query: ", query)
print ("Option:", "Logo")

for item in res["data"]:
  display(Image(url=item["thumbnail_url"]))
  print("Confidence Level:", item["confidence"])
  print("Confidence Score:", item["score"])

Search Query:  Subway
Option: Logo


## 4 - Classify API
Content classification is the process of organizing content into distinct categories based on specific criteria. This process helps you organize your videos into more manageable and useful categories, making them easier to find, access, and use.

If you have an extensive library of videos you'd like to analyze to identify specific actions or entities, manually finding the segments that meet your criteria is time-consuming. With a single API call, the Twelve Labs platform automatically identifies the actions and entities you specify whenever they appear in your videos and assigns a likelihood score that indicates how likely those actions or entities are to occur in each of your videos.

The platform allows you to define either a simple or a hierarchical taxonomy system to organize your videos.  A simple taxonomy system is composed of a single set of categories, similar to [YouTube video categories](https://developers.google.com/youtube/v3/docs/videoCategories/list). A hierarchical taxonomy system is composed of categories and subcategories, similar to [IAB Tech Lab Content Taxonomy](https://iabtechlab.com/standards/content-taxonomy/).

The following parameters allow you to control how classification works:

- `classes`: An array of objects containing the names and definitions of the entities or actions that the platform must identify. Each object is composed of the following fields:
  - `name`: A string representing the name you want to give this class.
  - `prompts`: An array of strings that specifies what the class contains. The platform uses the values you provide in this array to classify your videos.
- `threshold`: A numerical value that indicates the confidence level of the video segment's match with a certain class.
- `ratio`: A numerical value that displays the ratio of the sum of the lengths of the matching video segments divided by the total length of the video.

### Classifying videos based on one class
The following example code classifies the content of your videos based on one classes - `BeautyTok`:

In [49]:
# Construct the URL of the `/classify/bulk` endpoint
CLASSIFY_BULK_URL = f"{API_URL}/classify/bulk"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data =  {
  "options": [
        "visual",
        "conversation",
        "text_in_video"
  ],  # Specify how the platform will analyze your video
  "index_id": INDEX_ID,  # Indicate the unique identifier of your index
  "classes": [
      {
          "name": "BeautyTok",
          "prompts": [
              "Makeup",
              "Skincare",
              "Cosmetic Products",
              "Doing Nails",
              "Doing Hair",
              "Eyeshadow",
              "Lipstick",
              "Blush"
          ]
      }
  ],
  "threshold": {
        "min_video_score": 80,  # Minimum video score is 80
        "min_duration_ratio": 0.5  # Minimum duration ratio is 50%
  }
}

# Make a classification request
response = requests.post(CLASSIFY_BULK_URL, headers=headers, json=data)
print (f'Status code: {response.status_code}')
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 0, 'page_expired_at': '', 'total_results': 0}}


In the example output above, note that the `data` array is composed of three objects. Each object contains the following:

*   A field named `video_id` representing the unique identifier of the video that has been classified.
*   An array named `labels` containing information about each video. Note that, when classifying a video, the API service finds all video fragments that match the label you've specified in the request. For each video fragment found, the API service determines the level of confidence that the fragment matches the label. The `max_score` field is determined by comparing the confidence scores of each fragment and selecting the highest one.

### Retrieving detailed information about each matching video fragment

The following example code sets the `include_clips` parameter to true to specify that the API service should retrieve detailed information about each matching video fragment:

In [50]:
# Construct the URL of the `/classify/bulk` endpoint
CLASSIFY_BULK_URL = f"{API_URL}/classify/bulk"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data =  {
  "options": [
        "visual",
        "conversation",
        "text_in_video"
  ],
  "index_id": INDEX_ID,
  "include_clips": True,
  "classes": [
      {
          "name": "BeautyTok",
          "prompts": [
              "Makeup",
              "Skincare",
              "Cosmetic Products",
              "Doing Nails",
              "Doing Hair",
              "Eyeshadow",
              "Lipstick",
              "Blush"
          ]
      }
  ],
  "threshold": {
        "min_video_score": 80,
        "min_duration_ratio": 0.5
  }
}

# Make a classification request
response = requests.post(CLASSIFY_BULK_URL, headers=headers, json=data)
print (f'Status code: {response.status_code}')
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 0, 'page_expired_at': '', 'total_results': 0}}


In the example output above, note that, for each video, the API service returns an array named `clips` containing detailed information about a single matching video fragment.

### Classifying a video based on multiple classes

The following example code classifies the content of your videos based on three classes - `BeautyTok`, `FashionTok`, and `CookTok`:

In [51]:
# Construct the URL of the `/classify/bulk` endpoint
CLASSIFY_BULK_URL = f"{API_URL}/classify/bulk"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data =  {
  "options": [
        "visual",
        "conversation",
        "text_in_video"
  ],
  "index_id": INDEX_ID,
  "classes": [
        {
          "name": "BeautyTok",
          "prompts": [
              "Makeup",
              "Skincare",
              "Cosmetic Products",
              "Doing Nails",
              "Doing Hair",
              "Eyeshadow",
              "Lipstick",
              "Blush"
          ]
      },
        {
            "name": "FashionTok",
            "prompts": [
                "Clothing",
                "Outfit",
                "Fashion Design",
                "Men's Fashion",
                "Women's Fashion",
                "Find Your Style",
                "Upcoming Fashion Trends",
                "Accessories",
                "Streetwear",
                "Jewelry",
                "Clothing Care",
                "Style Advice"
            ]
        },
        {
            "name": "CookTok",
            "prompts": [
                "Cooking Tutorial",
                "Cooking Utensils",
                "Baking Tutorials",
                "Recipes",
                "Restaurants",
                "Food",
                "Cooking Tips",
                "Cooking Techniques",
                "Healthy Cooking"
            ]
        }
  ],
  "threshold": {
        "min_video_score": 80,
        "min_duration_ratio": 0.5
  }
}

# Make a classification request
response = requests.post(CLASSIFY_BULK_URL, headers=headers, json=data)
print (f'Status code: {response.status_code}')
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 0, 'page_expired_at': '', 'total_results': 0}}


In the example output above, note that the `data` array contains three objects, each corresponding to a different video. For each video, the response contains information that helps you determine the likelihood that each of the labels you've specified in the request appears in that video.

### Retrieving a detailed score for each class

The following example code sets the `show_detailed_score` parameter to `true` to specify that the platform should retrieve the maximum score, average score, duration weighted score, and normalized score for each class:

In [52]:
# Construct the URL of the `/classify/bulk` endpoint
CLASSIFY_BULK_URL = f"{API_URL}/classify/bulk"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data =  {
  "options": ["visual"],
  "index_id": INDEX_ID,
  "show_detailed_score": True,
  "classes": [
        {
          "name": "BeautyTok",
          "prompts": [
              "Makeup",
              "Skincare",
              "Cosmetic Products",
              "Doing Nails",
              "Doing Hair",
              "Eyeshadow",
              "Lipstick",
              "Blush"
          ]
      },
        {
            "name": "FashionTok",
            "prompts": [
                "Clothing",
                "Outfit",
                "Fashion Design",
                "Men's Fashion",
                "Women's Fashion",
                "Find Your Style",
                "Upcoming Fashion Trends",
                "Accessories",
                "Streetwear",
                "Jewelry",
                "Clothing Care",
                "Style Advice"
            ]
        },
        {
            "name": "CookTok",
            "prompts": [
                "Cooking Tutorial",
                "Cooking Utensils",
                "Baking Tutorials",
                "Recipes",
                "Restaurants",
                "Food",
                "Cooking Tips",
                "Cooking Techniques",
                "Healthy Cooking"
            ]
        }
  ],
  "threshold": {
        "min_video_score": 80,
        "min_duration_ratio": 0.5
  }
}

# Make a classification request
response = requests.post(CLASSIFY_BULK_URL, headers=headers, json=data)
print (f'Status code: {response.status_code}')
pprint(response.json())

Status code: 200
{'data': [],
 'page_info': {'limit_per_page': 0, 'page_expired_at': '', 'total_results': 0}}


## 5 - Generate API
The Generate API suite creates concise textual representations such as titles, summaries, chapters, and highlights for your videos, to name as a few. Unlike conventional models limited to unimodal interpretations (for example, summary videos relying solely on video transcription), the Generate API suite uses a multimodal approach that analyzes the whole context of a video, including visuals, sounds, spoken words, and texts and their relationship with one another. This ensures a holistic understanding of your videos, capturing nuances that an unimodal interpretation might miss. And then, using video-to-text generative capabilities, our model can turn that understanding into actual text format of your choice.

The Generate API suite offers three distinct endpoints tailored to meet various requirements for creating concise textual representations from videos. Each endpoint has been designed with specific levels of flexibility and customization to accommodate different needs.

**1 - The `/gist` endpoint**

- **Function**: Generates topics, titles, and hashtags.
- **Customization**: Uses predefined templates.
- **Prompt **: No.
- **Best use**: To generate an immediate and straightforward text representation without specific customization.

**2 - The `/summarize` endpoint**

- **Function**: Generates summaries, chapters, and highlights.
- **Customization**: Operates primarily on predefined templates, similar to `/gist`. However, you can provide a custom prompt that guides the model on how to generate the output.
- **Prompt**: Optional. While you can invoke this endpoint without a prompt, providing one allows for tailored outputs.
- **Best use:** To balance the efficiency of built-in templates and bespoke customization abilities.

**3 - The `/generate` endpoint**

- **Function**: Generates open-ended text representations from videos.
- **Customization**: Relies solely on user-defined prompts, ensuring maximum flexibility.
- **Prompt**: Required. You must provide clear instructions to guide the model.
- **Best use**: Ideal for advanced users with specific output requirements beyond the built-in templates.


In [53]:
# Specify the unique identifier of an existing video ('Jenna Marbles - How Girls Get Dressed')
%env VIDEO_ID=6528b54a43e8c47e4eb47e80

env: VIDEO_ID=6528b54a43e8c47e4eb47e80


In [54]:
# Retrieve the unique identifier of the existing video
VIDEO_ID = os.getenv("VIDEO_ID")
print (VIDEO_ID)
assert VIDEO_ID

6528b54a43e8c47e4eb47e80


To demonstrate the Generate API, let's work with this video: [Jenna Marbles - How Girls Get Dressed](https://youtu.be/0Hc0Z6zFbzM?si=a1oqVb0Eve9yzxMD)

### 5.1 - Generate titles, topics, and hashtags
Use the `/gist` endpoint if you require a swift breakdown of the essence of your videos. This endpoint operates exclusively on a set of predefined templates to generate the following concise and accurate textual representations:

- **Title**: Distills the essence of a video into a brief, coherent phrase, facilitating quick understanding and categorization. For example, a title like "From Consumerism to Minimalism: A Journey Toward Sustainable Living" clearly indicates the video's narrative trajectory,  emphasizing themes related to consumption patterns and sustainable lifestyles.
- **Topic**: Represents the central themes or subjects of a video and provides a high-level understanding based on the meaning conveyed by all modalities. Topics enable effective categorization and quick referencing. For example, "Shopping Vlog Lifestyle" denotes a video covering aspects related to shopping experiences, vlogging, and the associated lifestyle.
- **Hashtag**: Concisely summarizes the themes, subjects, or sentiments expressed within a video. Hashtags improve categorization and searchability on social media platforms. For example, hashtags such as `#BlackFriday,` `#ShoppingMania,` and `#Consumerism` indicate key focal points in a video, potentially related to shopping events, consumer behaviors, or broader commentary on consumption patterns.

Handling this set of predefined tasks offers a streamlined approach to obtaining concise textual representations, essential for rapid content understanding and categorization.

The following example generates a title, topic, and hashtag for the specified video:

In [55]:
# Define the /gist endpoint
GIST_URL = f"{API_URL}/gist"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "types": ["title", "topic", "hashtag"]
}

# Make a gist request
response = requests.post(GIST_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


### 5.2 - Generate summaries, chapters, and highlights
Use the `/summarize` endpoint if you want to utilize pre-defined templates for general summarization tasks and, optionally, provide a prompt to customize the output. This endpoint generates the following textual representations:

- **Summaries**:  The platform returns a brief that encapsulates the key points of a video, presenting the most important information clearly and concisely. Depending on your prompt, a summary can take various forms, such as a single paragraph, a series of paragraphs, an email, or a structured list of bullet points. For example, a summary highlighting Black Friday events might include a description of a crowded mall, key commentary by a news reporter on consumer behavior, and individual perspectives on societal values associated with consumption.
- **Chapters**: The platform returns a chronological list of all the segments in a video, providing a granular breakdown of its content. For each chapter, the platform returns its starting and end times, measured in seconds from the beginning of the video segment, a descriptive headline that offers a brief of the events or activities within that segment, and an accompanying summary that elaborates on the headline. For example, the first chapter of a stand-up comedy might describe the comedian's entrance and the first joke. The accompanying summary could delve into the content, detailing the comedian's humorous take on a specific subject, such as the cultural nuances of Tai Chi exercises.
- **Highlights**: The platform returns a chronologically ordered list of the most important events within a video. Unlike chapters, highlights only capture the key moments, providing a snapshot of the video's main topics. For each highlight, the platform returns its starting and end times, measured in seconds from the beginning of the video, and a brief description that captures the essence of the segment. For example, a highlight might capture a significant event like a bank heist in a video with multiple scenes.

The following example code generates a summary for the specified video:

In [56]:
# Define the /summarize endpoint
SUMMARIZE_URL = f"{API_URL}/summarize"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "type": "summary"
}

# Make a summarization request
response = requests.post(SUMMARIZE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


Optionally, you can use the `prompt` field to provide context for the summarization task. The following example specifies the purpose and the desired length:

In [57]:
# Define the /summarize endpoint
SUMMARIZE_URL = f"{API_URL}/summarize"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "type": "summary",
  "prompt": "Generate a summary of this video for a fashion show, up to three sentences."
}

# Make a summarization request
response = requests.post(SUMMARIZE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


The following example generates a list of chapters for the specified video:

In [58]:
# Define the /summarize endpoint
SUMMARIZE_URL = f"{API_URL}/summarize"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "type": "chapter"
}

# Make a summarization request
response = requests.post(SUMMARIZE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


Optionally, you can use the `prompt` parameter to indicate that the tone of voice should be stressful:

In [59]:
# Define the /summarize endpoint
SUMMARIZE_URL = f"{API_URL}/summarize"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "type": "chapter",
  "prompt": "Generate chapters using stressful language."
}

# Make a summarization request
response = requests.post(SUMMARIZE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


The following example generates a list of the important events or activities within a video:

In [60]:
# Define the /summarize endpoint
SUMMARIZE_URL = f"{API_URL}/summarize"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "type": "highlight"
}

# Make a summarization request
response = requests.post(SUMMARIZE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


Optionally, you can use the `prompt` parameter to generate highlights for the same video, showcasing the most confusing parts:

In [61]:
# Define the /summarize endpoint
SUMMARIZE_URL = f"{API_URL}/summarize"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "type": "highlight",
  "prompt": "Generate highlights that showcase the most confusing parts of the video."
}

# Make a summarization request
response = requests.post(SUMMARIZE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


### 5.3 - Generate open-ended textual representations
Use the `/generate` endpoint for open-ended textual representations that are more customizable and tailor-made than the templates offered by the `/gist` and `/summarize` endpoints. This endpoint can produce a diverse range of textual representations based on your prompt, including, but not limited to, tables of content, action items, memos, reports and comprehensive analyses.

The following example generates an Instagram post about women's fashion choice:

In [62]:
# Define the /generate endpoint
GENERATE_URL = f"{API_URL}/generate"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "prompt": "Write an Instagram post about women's fashion choice based on this video."
}

# Make a generation request
response = requests.post(GENERATE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


The following example generates a rap song based on the provided template:

In [63]:
# Define the /generate endpoint
GENERATE_URL = f"{API_URL}/generate"

# Set the header of the request
headers = {
    "x-api-key": API_KEY
}

# Declare a dictionary named `data`
data = {
  "video_id": VIDEO_ID,
  "prompt": "Write a rap song based on this video."
}

# Make a generation request
response = requests.post(GENERATE_URL, headers=headers, json=data)
print(f"Status code: {response.status_code}")
pprint(response.json())

Status code: 404
{'code': 'resource_not_exists',
 'message': "Resource with id '6528b54a43e8c47e4eb47e80' does not exist in "
            'video collection.'}


Have fun playing around with the Twelve Labs API!🚀